In [25]:
!pip install qiskit

In [26]:
!pip show qiskit

Name: qiskit
Version: 0.44.2
Summary: Software for developing quantum computing programs
Home-page: https://qiskit.org/
Author: Qiskit Development Team
Author-email: hello@qiskit.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: qiskit-terra
Required-by: 


In [27]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, transpile
from qiskit.quantum_info.operators import Operator
import numpy as np

In [28]:
from qiskit.algorithms import AmplificationProblem, Grover
from qiskit.quantum_info import Statevector
from qiskit.primitives import Sampler

In [34]:
!pip install qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 121.9 MB/s eta 0:00:00


In [47]:
def abs_list(ls):
  new_ls = [0]*len(ls)
  for i in range(len(ls)):
    new_ls[i] = abs(ls[i])
  return new_ls

In [48]:
"""
    We are creating a custom oracle that has only one solution state, which is
    determined by a minimum number. To accomplish this, we can utilize an Operator
    class that allows us to abstract away from the specific implementation of the
    gate used to mark the desired solution state.
"""
def create_oracle(nqubits: int, min_number: int):
    qr = QuantumRegister(nqubits)
    qc = QuantumCircuit(qr, name='Oracle(max)')

    gate_matrix = np.eye(2 ** nqubits, dtype=int)
    i=abs(min_number)
    gate_matrix[i][i] = -1

    min_operator = Operator(gate_matrix)
    qc.unitary(min_operator, range(nqubits))

    return qc.to_instruction()

def create_diffuser(nqubits: int):
    qr = QuantumRegister(nqubits)
    qc = QuantumCircuit(qr, name='Diffuser')

    for qubit in range(nqubits):
        qc.h(qubit)
    for qubit in range(nqubits):
        qc.x(qubit)
    qc.barrier(range(nqubits))
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)
    qc.h(nqubits-1)
    qc.barrier(range(nqubits))
    for qubit in range(nqubits):
        qc.x(qubit)
    for qubit in range(nqubits):
        qc.h(qubit)

    return qc.to_instruction()

def find_the_smallest_number(list_of_numbers):
    list_of_numbers = tuple(list_of_numbers)
    min_number = min(list_of_numbers)
    abs_ls = abs_list(list_of_numbers)
    nqubits = max(max(abs_ls).bit_length(), 2)
    nstates = 2 ** nqubits

    qc = QuantumCircuit(nqubits)

    for i in range(nqubits):
        qc.h(i)

    niterations = int(np.floor(np.pi/4 * np.sqrt(nstates)))
    for i in range(niterations):
        qc.append(create_oracle(nqubits, min_number), range(nqubits))
        qc.append(create_diffuser(nqubits), range(nqubits))

    qc.measure_all()
    qc.draw()
    aer_sim = Aer.get_backend('aer_simulator')
    compiled_circuit = transpile(qc, aer_sim)
    result = aer_sim.run(compiled_circuit).result()
    counts = result.get_counts()
    min_candidate = int(max(counts, key=counts.get), 2)

    if abs(min_number) == min_candidate:
        return min_number
    else:
        return None
def find_negative_numbers(list_of_numbers):
  min_number = find_the_smallest_number(list_of_numbers)
  print(min_number<0)

In [49]:
find_the_smallest_number([5,-6,-77,8,9])

-77

In [50]:
find_negative_numbers([5,-6,-77,8,9])

True
